# Setup file

List of every code snippet checking for a specific configuration :
- Run each code block individually
- If none of them cause an error when executed, the setup is ready to go
- Ignore cuda & cudNN warnings by tensorflow

## Documentation links

[Tensorflow guide](https://www.tensorflow.org/guide)  
[Keras guide](https://keras.io/guides/)  

## Python libraries

### Libraries

In [1]:
import pandas
import numpy
import tensorflow
import matplotlib
import keras_tuner
import seaborn
import graphviz
import pydot
import scikeras
import sklearn
import os

2024-02-03 05:14:37.380852: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 05:14:37.380927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 05:14:37.381846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-03 05:14:37.387988: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-03 05:14:38.072307: W tensorflow/compiler/tf2

### Dependencies

Most of those imports check if any library version has been changed (example : keras.utils.vis_utils has been moved to keras.utils)

In [2]:
from tensorflow import *

from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Input, GlobalAveragePooling1D, Dropout
from keras.preprocessing import timeseries_dataset_from_array
from keras.utils import plot_model

from scikeras.wrappers import KerasRegressor

from matplotlib import pyplot

from sklearn.preprocessing import normalize, LabelBinarizer
from sklearn.model_selection import train_test_split

## Tensorflow GPU

Useful setup guide : [Link](https://www.tensorflow.org/guide/gpu)

Expected output of the following code : 

    [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
     
    <function is_built_with_cuda at 0x7f4f5730fbf8>
     
    /device:GPU:0
     
    [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
    Cuda & CudNN warnings

In [3]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))
print("\n")
print(tf.test.is_built_with_cuda)
print("\n")
print(tf.test.gpu_device_name())
print("\n")
print(tf.config.get_visible_devices())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


<function is_built_with_cuda at 0x7f41c473dd80>


/device:GPU:0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-02-03 05:14:39.386818: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 05:14:39.566572: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 05:14:39.566803: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## TensorBoard

### TensorBoard Notebook loading

In [4]:
%load_ext tensorboard

### Tensorboard loading test

In [5]:
# Wipe the directory full of test logs
os.system("rm -rf ./logs/")

0

In [6]:
import tensorflow as tf
import datetime

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])


model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])


%tensorboard --logdir logs/fit

2024-02-03 05:14:40.772427: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/5


2024-02-03 05:14:42.217787: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f4086127540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-03 05:14:42.217821: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti with Max-Q Design, Compute Capability 7.5
2024-02-03 05:14:42.231545: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-03 05:14:42.264466: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1706933682.352673   19978 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 513/1875 [=======>......................] - ETA: 4s - loss: 0.3743 - accuracy: 0.8900

KeyboardInterrupt: 

## File pathing setup


### Configuration variables

In [9]:
# Parameter
wipe_dirs=True

# Sub-directories
data_dir_sub=["temp","zip","feather"]
result_dir_sub=["search","models","tensorboard"]

### Tests

In [10]:
# Get current directory
FILENAME="env_setup.ipynb"
DIRNAME=os.path.abspath(FILENAME).replace(FILENAME,'')

if os.path.isdir(DIRNAME+"data/"):
    for dir in data_dir_sub:
        if not os.path.isdir(DIRNAME+"data/"+dir+"/"):
            os.system("mkdir "+DIRNAME+"data/"+dir)
        elif wipe_dirs:
            os.system('rm -r '+DIRNAME+'data/'+dir+"/*")
else :
    os.system("mkdir "+DIRNAME+"data")

if os.path.isdir(DIRNAME+"results/"):
    for dir in result_dir_sub:
        if not os.path.isdir(DIRNAME+"results/"+dir+"/"):
            os.system("mkdir "+DIRNAME+"results/"+dir)
        elif wipe_dirs:
            os.system('rm -r '+DIRNAME+'results/'+dir+"/*")
else :
    os.system("mkdir "+DIRNAME+"results")

if not os.path.isdir(DIRNAME+"config/"):
    os.system("mkdir "+DIRNAME+"config")

rm: cannot remove '/home/quentin/Documents/ai_force_torque/data/temp/*': No such file or directory
rm: cannot remove '/home/quentin/Documents/ai_force_torque/data/zip/*': No such file or directory
rm: cannot remove '/home/quentin/Documents/ai_force_torque/results/search/*': No such file or directory
rm: cannot remove '/home/quentin/Documents/ai_force_torque/results/models/*': No such file or directory
